In [13]:
import sys
sys.path.append('../')
import os
import collections
import random
from pathlib import Path
import logging
import shutil
import time
from packaging import version
from collections import defaultdict

from tqdm import tqdm
import numpy as np
import gzip
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.backends.cudnn as cudnn

from src.param import parse_args
from src.utils import LossMeter
from src.dist_utils import reduce_dict
from transformers import T5Tokenizer, T5TokenizerFast
from src.tokenization import P5Tokenizer, P5TokenizerFast
from src.pretrain_model import P5Pretraining

_use_native_amp = False
_use_apex = False

# Check if Pytorch version >= 1.6 to switch between Native AMP and Apex
if version.parse(torch.__version__) < version.parse("1.6"):
    from transormers.file_utils import is_apex_available
    if is_apex_available():
        from apex import amp
    _use_apex = True
else:
    _use_native_amp = True
    from torch.cuda.amp import autocast

from src.trainer_base import TrainerBase

import pickle

def load_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        
import json

def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)
    
def ReadLineFromFile(path):
    lines = []
    with open(path,'r') as fd:
        for line in fd:
            lines.append(line.rstrip('\n'))
    return lines

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [14]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self
        
args = DotDict()

args.distributed = False
args.multiGPU = True
args.fp16 = True
args.train = "beauty"
args.valid = "beauty"
args.test = "beauty"
args.batch_size = 16
args.optim = 'adamw' 
args.warmup_ratio = 0.05
args.lr = 1e-3
args.num_workers = 4
args.clip_grad_norm = 1.0
args.losses = 'rating,sequential,explanation,review,traditional'
args.backbone = 't5-small' # small or base
args.output = 'snap/beauty-small'
args.epoch = 10
args.local_rank = 0

args.comment = ''
args.train_topk = -1
args.valid_topk = -1
args.dropout = 0.1

args.tokenizer = 'p5'
args.max_text_length = 512
args.do_lower_case = False
args.word_mask_rate = 0.15
args.gen_max_length = 64

args.weight_decay = 0.01
args.adam_eps = 1e-6
args.gradient_accumulation_steps = 1
args.continuous_embed = True
'''
Set seeds
'''
args.seed = 2022
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

'''
Whole word embedding
'''
args.whole_word_embed = True

cudnn.benchmark = True
ngpus_per_node = torch.cuda.device_count()
args.world_size = ngpus_per_node

LOSSES_NAME = [f'{name}_loss' for name in args.losses.split(',')]
if args.local_rank in [0, -1]:
    print(LOSSES_NAME)
LOSSES_NAME.append('total_loss') # total loss

args.LOSSES_NAME = LOSSES_NAME

gpu = 0 # Change GPU ID
args.gpu = gpu
args.rank = gpu
print(f'Process Launching at GPU {gpu}')

torch.cuda.set_device('cuda:{}'.format(gpu))

comments = []
dsets = []
if 'toys' in args.train:
    dsets.append('toys')
if 'beauty' in args.train:
    dsets.append('beauty')
if 'sports' in args.train:
    dsets.append('sports')
comments.append(''.join(dsets))
if args.backbone:
    comments.append(args.backbone)
comments.append(''.join(args.losses.split(',')))
if args.comment != '':
    comments.append(args.comment)
comment = '_'.join(comments)

if args.local_rank in [0, -1]:
    print(args)

['rating_loss', 'sequential_loss', 'explanation_loss', 'review_loss', 'traditional_loss']
Process Launching at GPU 0
{'distributed': False, 'multiGPU': True, 'fp16': True, 'train': 'beauty', 'valid': 'beauty', 'test': 'beauty', 'batch_size': 16, 'optim': 'adamw', 'warmup_ratio': 0.05, 'lr': 0.001, 'num_workers': 4, 'clip_grad_norm': 1.0, 'losses': 'rating,sequential,explanation,review,traditional', 'backbone': 't5-small', 'output': 'snap/beauty-small', 'epoch': 10, 'local_rank': 0, 'comment': '', 'train_topk': -1, 'valid_topk': -1, 'dropout': 0.1, 'tokenizer': 'p5', 'max_text_length': 512, 'do_lower_case': False, 'word_mask_rate': 0.15, 'gen_max_length': 64, 'weight_decay': 0.01, 'adam_eps': 1e-06, 'gradient_accumulation_steps': 1, 'continuous_embed': True, 'seed': 2022, 'whole_word_embed': True, 'world_size': 2, 'LOSSES_NAME': ['rating_loss', 'sequential_loss', 'explanation_loss', 'review_loss', 'traditional_loss', 'total_loss'], 'gpu': 0, 'rank': 0}


In [15]:
def create_config(args):
    from transformers import T5Config, BartConfig

    if 't5' in args.backbone:
        config_class = T5Config
    else:
        return None

    config = config_class.from_pretrained(args.backbone)
    config.dropout_rate = args.dropout
    config.dropout = args.dropout
    config.attention_dropout = args.dropout
    config.activation_dropout = args.dropout
    config.losses = args.losses

    return config


def create_tokenizer(args):
    from transformers import T5Tokenizer, T5TokenizerFast
    from src.tokenization import P5Tokenizer, P5TokenizerFast

    if 'p5' in args.tokenizer:
        tokenizer_class = P5Tokenizer

    tokenizer_name = args.backbone
    
    if args.continuous_embed:
        datamaps = load_json(os.path.join('../data', args.train, 'datamaps.json'))
        user_extra_ids = len(datamaps['user2id']) + 1 # index start from 1
        item_extra_ids = len(datamaps['item2id']) + 1 
    else:
        user_extra_ids = 0
        item_extra_ids = 0
    tokenizer = P5Tokenizer.from_pretrained(
            args.backbone, 
            max_length=args.max_text_length, 
            do_lower_case=args.do_lower_case,
            user_extra_ids=user_extra_ids,
            item_extra_ids=item_extra_ids
            )

    print(tokenizer_class, tokenizer_name)
    
    return tokenizer


def create_model(model_class, config=None):
    print(f'Building Model at GPU {args.gpu}')

    model_name = args.backbone

    model = model_class.from_pretrained(
        model_name,
        config=config
    )
    return model

In [16]:
config = create_config(args)

if args.tokenizer is None:
    args.tokenizer = args.backbone
    
tokenizer = create_tokenizer(args)

model_class = P5Pretraining
model = create_model(model_class, config)

model = model.cuda()

if 'p5' in args.tokenizer:
    model.resize_token_embeddings(tokenizer.vocab_size)
    
model.tokenizer = tokenizer

<class 'src.tokenization.P5Tokenizer'> t5-small
Building Model at GPU 0


Some weights of P5Pretraining were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.whole_word_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Load Model

In [17]:
args.load = "../snap/beauty-small/Epoch10.pth"

# Load Checkpoint
from src.utils import load_state_dict, LossMeter, set_global_logging_level
from pprint import pprint

def load_checkpoint(ckpt_path):
    state_dict = load_state_dict(ckpt_path, 'cpu')
    results = model.load_state_dict(state_dict, strict=False)
    print('Model loaded from ', ckpt_path)
    pprint(results)

ckpt_path = args.load
load_checkpoint(ckpt_path)

from src.all_amazon_templates import all_tasks as task_templates

Model loaded from  ../snap/beauty-small/Epoch10.pth
<All keys matched successfully>


#### Check Test Split

In [18]:
data_splits = load_pickle('../data/beauty/rating_splits_augmented.pkl')
test_review_data = data_splits['test']

In [19]:
len(test_review_data)

19850

In [20]:
test_review_data[0]

{'reviewerID': 'A2QKXW3LDQ66P5',
 'asin': 'B005X2F7KI',
 'reviewerName': 'stephanie',
 'helpful': [5, 6],
 'reviewText': 'Absolutely great product.  I bought this for my fourteen year old niece for Christmas and of course I had to try it out, then I tried another one, and another one and another one.  So much fun!  I even contemplated keeping a few for myself!',
 'overall': 5.0,
 'summary': 'Perfect!',
 'unixReviewTime': 1352937600,
 'reviewTime': '11 15, 2012',
 'explanation': 'Absolutely great product',
 'feature': 'product'}

In [21]:
data_maps = load_json(os.path.join('../data', 'beauty', 'datamaps.json'))
print(len(data_maps['user2id'])) # number of users
print(len(data_maps['item2id'])) # number of items

22363
12101


### Test P5

In [22]:
from torch.utils.data import DataLoader, Dataset, Sampler
from src.pretrain_data import get_loader
from evaluate.utils import rouge_score, bleu_score, unique_sentence_percent, root_mean_square_error, mean_absolute_error, feature_detect, feature_matching_ratio, feature_coverage_ratio, feature_diversity
from evaluate.metrics4rec import evaluate_all

#### Evaluation - Rating

In [23]:
test_task_list = {'rating': ['1-10'] # or '1-6'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), float(p)) for (r, p) in zip(gt_ratings, pred_ratings) if p in [str(i/10.0) for i in list(range(10, 50))]]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

66566
Data sources:  ['beauty']
1241


1241it [00:31, 39.45it/s]


RMSE  1.2876
MAE  0.8461


In [30]:
test_task_list = {'rating': ['1-1'] # or '1-10'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
predicted_rating = [(float(r), float(p)) for (r, p) in zip(gt_ratings, pred_ratings) if p in [str(i/10.0) for i in list(range(10, 50))]]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

66566
Data sources:  ['beauty']
1241


1241it [00:31, 39.92it/s]


RMSE  1.2803
MAE  0.8499


#### Evaluation - Sequential

In [31]:
test_task_list = {'sequential': ['2-13'] # or '2-3'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1][6:-1] #<item_[num]> to [num]
            new_info['gen_item_list'] = [item[6:-1] for item in generated_sents[j*20: (j+1)*20]]
            all_info.append(new_info)
# gt = {}
# ui_scores = {}
# for i, info in enumerate(all_info):
#     gt[i] = [int(info['target_item'])]
#     pred_dict = {}
#     for j in range(len(info['gen_item_list'])):
#         try:
#             pred_dict[int(info['gen_item_list'][j])] = -(j+1)
#         except:
#             pass
#     ui_scores[i] = pred_dict
    
# evaluate_all(ui_scores, gt, 5)
# evaluate_all(ui_scores, gt, 10)

66566
Data sources:  ['beauty']
1398


1398it [04:52,  4.78it/s]


In [32]:
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict #key:candidate item, val rank
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)


NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0247	0.0349	0.0349	0.0070	0.0213	0.0213

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0295	0.0498	0.0498	0.0050	0.0233	0.0233


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0295\t0.0498\t0.0498\t0.0050\t0.0233\t0.0233',
 {'ndcg': 0.029484483245804716,
  'map': 0.023299167630205505,
  'recall': 0.04981442561373697,
  'precision': 0.004981442561373606,
  'mrr': 0.023299167630205505,
  'hit': 0.04981442561373697})

In [33]:
test_task_list = {'sequential': ['2-3'] # or '2-13'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1][6:-1] #<item_[num]> to [num]
            new_info['gen_item_list'] = [item[6:-1] for item in generated_sents[j*20: (j+1)*20]]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

66566
Data sources:  ['beauty']
1398


1398it [04:51,  4.79it/s]



NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0248	0.0352	0.0352	0.0070	0.0213	0.0213

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0296	0.0502	0.0502	0.0050	0.0233	0.0233


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0296\t0.0502\t0.0502\t0.0050\t0.0233\t0.0233',
 {'ndcg': 0.029583526563089175,
  'map': 0.023306336501122533,
  'recall': 0.05017215937038859,
  'precision': 0.0050172159370387645,
  'mrr': 0.023306336501122533,
  'hit': 0.05017215937038859})

#### Evaluation - Explanation

In [34]:
test_task_list = {'explanation': ['3-12'] # or '3-9' or '3-3'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                min_length=9,
                num_beams=12,
                num_return_sequences=1,
                num_beam_groups=3,
                repetition_penalty=0.7
        )
        results = model.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
new_tokens_predict = [l.split() for l in tokens_predict]
new_tokens_test = [ll.split() for ll in tokens_test]
BLEU1 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=1, smooth=False)
BLEU4 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=4, smooth=False)
ROUGE = rouge_score(tokens_test, tokens_predict)

print('BLEU-1 {:7.4f}'.format(BLEU1))
print('BLEU-4 {:7.4f}'.format(BLEU4))
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

66566
Data sources:  ['beauty']
839


839it [06:48,  2.05it/s]


BLEU-1 19.5952
BLEU-4  2.9786
rouge_1/f_score 24.2635
rouge_1/r_score 25.7167
rouge_1/p_score 27.9660
rouge_2/f_score  5.5755
rouge_2/r_score  6.5761
rouge_2/p_score  6.2439
rouge_l/f_score 17.8403
rouge_l/r_score 23.0579
rouge_l/p_score 22.0192


In [35]:
test_task_list = {'explanation': ['3-9'] # or '3-12' or '3-3'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test,
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                min_length=10,
                num_beams=12,
                num_return_sequences=1,
                num_beam_groups=3
        )
        results = model.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
new_tokens_predict = [l.split() for l in tokens_predict]
new_tokens_test = [ll.split() for ll in tokens_test]
BLEU1 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=1, smooth=False)
BLEU4 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=4, smooth=False)
ROUGE = rouge_score(tokens_test, tokens_predict)

print('BLEU-1 {:7.4f}'.format(BLEU1))
print('BLEU-4 {:7.4f}'.format(BLEU4))
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

66566
Data sources:  ['beauty']
839


839it [06:38,  2.11it/s]


BLEU-1 20.4096
BLEU-4  3.0107
rouge_1/f_score 24.4934
rouge_1/r_score 27.0069
rouge_1/p_score 26.7931
rouge_2/f_score  5.3577
rouge_2/r_score  6.5417
rouge_2/p_score  5.7315
rouge_l/f_score 18.2630
rouge_l/r_score 23.2397
rouge_l/p_score 22.1677


In [36]:
test_task_list = {'explanation': ['3-3'] # or '3-12' or '3-9'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                min_length=10
        )
        results = model.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
new_tokens_predict = [l.split() for l in tokens_predict]
new_tokens_test = [ll.split() for ll in tokens_test]
BLEU1 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=1, smooth=False)
BLEU4 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=4, smooth=False)
ROUGE = rouge_score(tokens_test, tokens_predict)

print('BLEU-1 {:7.4f}'.format(BLEU1))
print('BLEU-4 {:7.4f}'.format(BLEU4))
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

66566
Data sources:  ['beauty']
839


839it [01:25,  9.78it/s]


BLEU-1 16.0193
BLEU-4  1.0376
rouge_1/f_score 16.8361
rouge_1/r_score 18.4659
rouge_1/p_score 17.9957
rouge_2/f_score  1.9903
rouge_2/r_score  2.3267
rouge_2/p_score  2.1658
rouge_l/f_score 12.2331
rouge_l/r_score 15.2562
rouge_l/p_score 14.5023


#### Evaluation - Review

Since T0 & GPT-2 checkpoints hosted on Hugging Face platform are slow to conduct inference, we only perform evaluation on the first 800 instances for prompts in Task Family 4.

In [37]:
test_task_list = {'review': ['4-4'] # or '4-2'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    if i > 50:
        break
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), round(float(p))) for (r, p) in zip(gt_ratings, pred_ratings)]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

66566
Data sources:  ['beauty']
1241


51it [00:01, 25.89it/s]


RMSE  0.6043
MAE  0.2868


In [38]:
test_task_list = {'review': ['4-2'] # or '4-4'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    if i > 50:
        break
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), round(float(p))) for (r, p) in zip(gt_ratings, pred_ratings)]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

66566
Data sources:  ['beauty']
1241


51it [00:01, 26.17it/s]


RMSE  0.6114
MAE  0.2880


In [39]:
test_task_list = {'review': ['4-1']
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    if i > 50:
        break
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
new_tokens_predict = [l.split() for l in tokens_predict]
new_tokens_test = [ll.split() for ll in tokens_test]
BLEU2 = bleu_score(new_tokens_test, new_tokens_predict, n_gram=2, smooth=False)
ROUGE = rouge_score(tokens_test, tokens_predict)

print('BLEU-2 {:7.4f}'.format(BLEU2))
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

66566
Data sources:  ['beauty']
1241


51it [00:03, 13.28it/s]


BLEU-2  3.1364
rouge_1/f_score  9.3585
rouge_1/r_score  8.7449
rouge_1/p_score 11.6387
rouge_2/f_score  2.0932
rouge_2/r_score  1.9822
rouge_2/p_score  2.4612
rouge_l/f_score  8.4988
rouge_l/r_score  8.7138
rouge_l/p_score 11.5803


#### Evaluation - Traditional

In [40]:
test_task_list = {'traditional': ['5-8']  # or '5-5'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1][6:-1] #<item_[num]> to [num]
            new_info['gen_item_list'] = [item[6:-1] for item in generated_sents[j*20: (j+1)*20]]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

66566
Data sources:  ['beauty']
1398


1398it [06:52,  3.39it/s]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0558	0.0558	0.0558	0.0558	0.0558	0.0558

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0997	0.1388	0.1388	0.0278	0.0868	0.0868

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1090	0.1671	0.1671	0.0167	0.0906	0.0906


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1090\t0.1671\t0.1671\t0.0167\t0.0906\t0.0906',
 {'ndcg': 0.10897694524261684,
  'map': 0.09062267322227971,
  'recall': 0.16710638107588427,
  'precision': 0.016710638107589163,
  'mrr': 0.09062267322227971,
  'hit': 0.16710638107588427})

In [41]:
test_task_list = {'traditional': ['5-5']  # or '5-8'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        tokenizer = tokenizer,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1][6:-1] #<item_[num]> to [num]
            new_info['gen_item_list'] = [item[6:-1] for item in generated_sents[j*20: (j+1)*20]]
            # print(item[2])
            # print(generated_sents[j*20: (j+1)*20])
            # print(new_info['gen_item_list'])
            # print(item[1])
            # print(new_info['target_item'])
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

66566
Data sources:  ['beauty']
1398


1398it [06:53,  3.38it/s]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0612	0.0612	0.0612	0.0612	0.0612	0.0612

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.1104	0.1552	0.1552	0.0310	0.0955	0.0955

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1202	0.1855	0.1855	0.0186	0.0996	0.0996


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1202\t0.1855\t0.1855\t0.0186\t0.0996\t0.0996',
 {'ndcg': 0.12022236309596888,
  'map': 0.09959896910216634,
  'recall': 0.1855296695434423,
  'precision': 0.018552966954345385,
  'mrr': 0.09959896910216634,
  'hit': 0.1855296695434423})